# Testing UNET Functionality 

In [1]:
import torch
from Utilities.Unet import Unet

# Define input tensor
input_tensor = torch.randn(1, 5, 144, 240)

# Define block dimensions
block_dimensions = [5, 128, 256, 512]

# Set output channels
output_channels = 128

# Adjusted up block concatenation flags and skip connection indices
up_block_use_concat = [False, True, False]  # Length matches the number of up blocks
skip_connection_indices = [1]  # Adjusted to match the number of True values

# Initialize the Unet model
model = Unet(
    block_dimensions=block_dimensions,
    output_channels=output_channels,
    up_block_use_concat=up_block_use_concat,
    skip_connection_indices=skip_connection_indices
)

# Forward pass
output = model(input_tensor)

# Check the output shape
print(output.shape)  # Should be [1, 128, 256, 256]


Initial Input Shape: torch.Size([1, 5, 144, 240])
After Initial DoubleConv: torch.Size([1, 5, 144, 240])
DownBlock Input Shape: torch.Size([1, 5, 144, 240])
After DoubleConv DownBlock: torch.Size([1, 128, 144, 240])
After Pooling DownBlock: torch.Size([1, 128, 72, 120])
DownBlock Input Shape: torch.Size([1, 128, 72, 120])
After DoubleConv DownBlock: torch.Size([1, 256, 72, 120])
After Pooling DownBlock: torch.Size([1, 256, 36, 60])
DownBlock Input Shape: torch.Size([1, 256, 36, 60])
After DoubleConv DownBlock: torch.Size([1, 512, 36, 60])
After Pooling DownBlock: torch.Size([1, 512, 18, 30])
UpBlock Input Shape: torch.Size([1, 512, 18, 30])
After UpConv: torch.Size([1, 256, 36, 60])
After DoubleConv UpBlock: torch.Size([1, 256, 36, 60])
UpBlock Input Shape: torch.Size([1, 256, 36, 60])
After UpConv: torch.Size([1, 128, 72, 120])
Skip Connection Shape: torch.Size([1, 256, 72, 120])
After Concatenation: torch.Size([1, 384, 72, 120])
After DoubleConv UpBlock: torch.Size([1, 128, 72, 120])

# Testing Spade Functionality

In [2]:
# Example usage of the SPADEGeneratorUnit

import torch
from Utilities.Spade import SPADEGeneratorUnit  # Ensure this path is correct

# Define parameters
in_channels = 5               # Number of channels in input feature map x
out_channels = 128             # Desired number of output features
mask_channels = 3              # Number of channels in the mask
kernel_size = 3                # Convolution kernel size
pad_mode = 'constant'          # Padding mode for F.pad (zero-padding)
upsampling = True              # Whether to apply upsampling

# Initialize the SPADEGeneratorUnit
spade_gen_unit = SPADEGeneratorUnit(
    in_channels=in_channels,
    out_channels=out_channels,
    mask_channels=mask_channels,
    kernel_size=kernel_size,
    padding_mode=pad_mode
)

# Move model to device (optional, but recommended)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
spade_gen_unit.to(device)

# Create dummy input tensors
batch_size = 8
height, width = 64, 64               # Spatial dimensions of input x
mask_height, mask_width = 64, 64     # Spatial dimensions of mask

# Input feature map x
x = torch.randn(batch_size, in_channels, height, width).to(device)

# Input mask
mask = torch.randn(batch_size, mask_channels, mask_height, mask_width).to(device)

# Forward pass through the SPADEGeneratorUnit with default noise behavior
output_default = spade_gen_unit(x, mask)
print("Output shape with default noise behavior:", output_default.shape)
# Expected Output Shape: [8, 128, 128, 128]

# Forward pass through the SPADEGeneratorUnit with noise explicitly enabled
output_with_noise = spade_gen_unit(x, mask, add_noise=True)
print("Output shape with noise enabled:", output_with_noise.shape)
# Expected Output Shape: [8, 128, 128, 128]

# Forward pass through the SPADEGeneratorUnit with noise explicitly disabled
output_without_noise = spade_gen_unit(x, mask, add_noise=False)
print("Output shape with noise disabled:", output_without_noise.shape)
# Expected Output Shape: [8, 128, 128, 128]


Output shape with default noise behavior: torch.Size([8, 128, 64, 64])
Output shape with noise enabled: torch.Size([8, 128, 64, 64])
Output shape with noise disabled: torch.Size([8, 128, 64, 64])


# Testing Resnet 

In [3]:
# Example usage of the ResNet and ResNetUnit
import torch
from Utilities.ResNet import ResNet

# Define parameters
in_channels = 3          # Number of input channels
block_dimensions = [64, 128, 256, 512]  # Feature dimensions for each residual block
kernel_size = 3          # Convolution kernel size
pooling = True           # Whether to apply max pooling after residual blocks
padding_mode = 'reflect' # Padding mode for convolutional layers
# Initialize the ResNet model
resnet = ResNet(
    in_channels=in_channels, 
    block_dimensions=block_dimensions, 
    kernel_size=kernel_size,
    pooling=pooling, 
    padding_mode=padding_mode
)
# Create a dummy input tensor
batch_size = 8
height, width = 128, 256   # Spatial dimensions of the input
x = torch.randn(batch_size, in_channels, height, width)
# Forward pass through the ResNet
output = resnet(x)
# Check the output shape
print("Final Output shape:", output.shape)  # Expected shape depends on pooling and block_dimensions





After Initial Conv: torch.Size([8, 64, 128, 256])
After ResNetBlock 1: torch.Size([8, 64, 128, 256])
After Pooling 1: torch.Size([8, 64, 64, 128])
After ResNetBlock 2: torch.Size([8, 128, 64, 128])
After Pooling 2: torch.Size([8, 128, 32, 64])
After ResNetBlock 3: torch.Size([8, 256, 32, 64])
After Pooling 3: torch.Size([8, 256, 16, 32])
After ResNetBlock 4: torch.Size([8, 512, 16, 32])
Final Output shape: torch.Size([8, 512, 16, 32])


# Testing Advection, Diffusion, Poisson & Finite Difference

In [4]:
# example_usage.py

import torch
import numpy as np
from Differentiator.Advection import Advection
from Differentiator.Diffusion import Diffusion
from Integrator.Poisson import Poisson  # Import Poisson from Integrator

def main():
    # Define parameters
    channel_size = 3  # Example: RGB channels
    cd_filter_1d = np.array([-1.0, 1.0])
    padding = "SYMMETRIC"

    # Instantiate the Advection, Diffusion, and Poisson layers
    advection_layer = Advection(channel_size=channel_size, 
                                cd_filter_1d=cd_filter_1d, 
                                padding_mode=padding)
    
    diffusion_layer = Diffusion(channel_size=channel_size, 
                                 cd_filter_1d=cd_filter_1d, 
                                 padding_mode=padding)
    
    poisson_layer = Poisson(channel_size=channel_size, 
                             cd_filter_1d=cd_filter_1d, 
                             padding_mode=padding)

    # Move layers to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    advection_layer.to(device)
    diffusion_layer.to(device)
    poisson_layer.to(device)

    # Create dummy inputs
    batch_size = 4
    C = channel_size
    H, W = 64, 64  # Spatial dimensions

    state_variable = torch.randn(batch_size, C, H, W).to(device)      # [N, C, H, W]
    velocity_field = torch.randn(batch_size, 2*C, H, W).to(device)  # [N, 2C, H-1, W-1]
    vector_field = [torch.randn(batch_size, C, H, W).to(device),   # First component
                    torch.randn(batch_size, C, H, W).to(device)]   # Second component

    # Forward pass through Advection layer
    advect = advection_layer(state_variable, velocity_field)
    print("Advection Output Shape:", advect.shape)  # Expected: [4, 1, 63, 63]

    # Forward pass through Diffusion layer
    laplacian = diffusion_layer(state_variable)
    print("Laplacian Output Shape:", laplacian.shape)  # Expected: [4, 3, 62, 62]

    # Forward pass through Poisson layer
    ux2, vy2, uyvx = poisson_layer(vector_field)
    print("Poisson Outputs Shapes:")
    print("ux2:", ux2.shape)   # Expected: [4, 3, 63, 63]
    print("vy2:", vy2.shape)   # Expected: [4, 3, 63, 63]
    print("uyvx:", uyvx.shape) # Expected: [4, 3, 63, 63]

if __name__ == "__main__":
    main()


Advection Output Shape: torch.Size([4, 1, 64, 64])
Laplacian Output Shape: torch.Size([4, 3, 64, 64])
Poisson Outputs Shapes:
ux2: torch.Size([4, 3, 64, 64])
vy2: torch.Size([4, 3, 64, 64])
uyvx: torch.Size([4, 3, 64, 64])


In [5]:
import torch
from Differentiator.MappingAndRecon import MappingAndRecon

def main():
    # Define parameters
    n_base_features = 128
    n_mask_channel = 2
    output_channel = 2
    padding_mode = "constant"  # Ensure lowercase

    # Instantiate the model
    model = MappingAndRecon(
        n_base_features=n_base_features,
        n_mask_channel=n_mask_channel,
        output_channel=output_channel,
        padding_mode=padding_mode
    )

    # Move model to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Create dummy inputs with arbitrary spatial dimensions
    batch_size = 4
    C = n_base_features
    M = n_mask_channel
    H, W = 128, 192  # Example spatial dimensions; can be changed as needed

    dynamic_feature = torch.randn(batch_size, C, H, W).to(device)      # [N, C, H, W]
    advec_diff = torch.randn(batch_size, M, H, W).to(device)          # [N, M, H, W]

    # Forward pass
    output = model(dynamic_feature, advec_diff)
    print("Output Shape:", output.shape)  # Expected: [4, 2, H_out, W_out]

if __name__ == "__main__":
    main()


After Initial Conv: torch.Size([4, 128, 128, 192])
After ResNetBlock 1: torch.Size([4, 128, 128, 192])
After ResNetBlock 2: torch.Size([4, 128, 128, 192])
Output Shape: torch.Size([4, 2, 128, 192])


In [6]:
# test_PoissonBlock.py

import torch
from Integrator.Poisson import PoissonBlock  # Ensure the import path is correct

# Define parameters
n_base_features = 64  # Number of base feature channels

# Initialize the PoissonBlock
poisson_block = PoissonBlock(n_base_features=n_base_features)

# Create a dummy input tensor
batch_size = 8
channels = 3  # Assuming 3 channels as per the original TensorFlow code
height, width = 128, 256  # Example spatial dimensions
x = torch.randn(batch_size, channels, height, width)

# Forward pass through the PoissonBlock
output = poisson_block(x)

# Check the output shape
print("Final Output shape:", output.shape)  # Expected: [8, 1, 128, 256]


DownBlock Input Shape: torch.Size([8, 6, 128, 256])
After DoubleConv DownBlock: torch.Size([8, 64, 128, 256])
After Pooling DownBlock: torch.Size([8, 64, 64, 128])
After Initial Conv: torch.Size([8, 64, 128, 256])
After ResNetBlock 1: torch.Size([8, 64, 128, 256])
After ResNetBlock 2: torch.Size([8, 64, 128, 256])
Final Output shape: torch.Size([8, 1, 128, 256])
